# 02.02 Geospatial Analysis: Census Blocks Only Visualizations
## Issue 1279
[Shapefile Source - 2020 US Census Blocks ](https://www2.census.gov/geo/tiger/TIGER2020PL/STATE/06_CALIFORNIA/06037/)

---

# Package & Data Imports
---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# https://stackoverflow.com/questions/13440102/getting-bounding-box-of-city
import geopandas as gpd

%matplotlib inline
import warnings
warnings.simplefilter("ignore")

In [ ]:
# https://geopandas.org/en/stable/getting_started/introduction.html
# NC boundaries: https://data.lacity.org/City-Infrastructure-Service-Requests/Neighborhood-Councils-Certified-/fu65-dz2f
raw_gdf_nc = gpd.read_file('../data/Neighborhood Councils (Certified)/geo_export_88bb18d9-f96c-4351-8be9-594f258ed0d3.shp')
gdf_nc = raw_gdf_nc.copy()

# Block boundaries: https://www2.census.gov/geo/tiger/TIGER2020PL/STATE/06_CALIFORNIA/06037/
# tl_2020_06037_tabblock20.zip
raw_gdf_blk = gpd.read_file("../data/tl_2020_06037_tabblock20/tl_2020_06037_tabblock20.shp")
gdf_blk = raw_gdf_blk.copy()

# merged nc and blk geodata
gdf_blk_nc_raw = gpd.read_file('../data/SHAPE-clean_01Oct21_01Oct22_nc_blk/clean_01Oct21_01Oct22_nc_blk.shp')
gdf_blk_nc = gdf_blk_nc_raw.copy()

# merged request dataset with nc block geodata
df_blk_nc_raw = gpd.read_file('../data/SHAPE-clean_01Oct21_01Oct22_nc_blk_req/clean_01Oct21_01Oct22_nc_blk_req.shp')
df_blk_nc = df_blk_nc_raw.copy()

# Block Choropleth
---

In [ ]:
gdf_blk_nc.head()

In [ ]:
df_blk_nc.head()

In [ ]:
df_blk_nc['GEOID20'].value_counts(ascending = False)

In [ ]:
# create groupby neighborhood council
df_blk_ct = df_blk_nc.groupby(['councilId','councilNam'])['GEOID20'].value_counts().reset_index(name = 'count')
print(df_blk_ct.shape)
df_blk_ct.head()

In [ ]:
# merge block grouping with geometries
gdf_blk_plt = pd.merge(gdf_blk_nc, df_blk_ct, how = 'inner', on = 'GEOID20')

# incase need to minimize dataset for testing
# gdf_blk_plt = gdf_blk_plt.sample(frac = .01, random_state = 42)

# sort and remove unnecessary columns
gdf_blk_plt = gdf_blk_plt.sort_values(by = 'GEOID20').reset_index(drop = True)
gdf_blk_plt = gdf_blk_plt[['GEOID20', 'geometry', 'councilId_x', 'name', 'count']]
gdf_blk_plt.rename(columns = {'councilId_x':'councilId', 'name_x':'councilName'}, inplace = True)

# check geodataframe
print(gdf_blk_plt.shape)
gdf_blk_plt.head()

In [ ]:
# check top count to compare to mapping
gdf_blk_plt.sort_values(by = 'count', ascending = False).head()

In [ ]:
gdf_blk_plt.groupby('name')['count'].sum().sort_values(ascending = False)

In [ ]:
# basic geopandas plot for comparison to folium
gdf_blk_plt.plot(column = 'count', scheme='percentiles', cmap='YlOrRd', figsize = (5, 10));

In [ ]:
# https://geopandas.org/en/stable/gallery/polygon_plotting_with_folium.html

# block basemap
fig = folium.Figure(height = 600, width = 1000)
blk_map = folium.Map(location = (34.03, -118.39), tiles='cartodbpositron', zoom_start=9.5)
fig.add_child(blk_map)

# block-by-block requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
bins = gdf_blk_plt["count"].quantile([0, 0.25, 0.5, 0.6, .7, .8, .9, .95, .98, 1]).tolist()

choro_blk = folium.Choropleth(
    geo_data = gdf_blk_plt.to_json(),
    name="Block Choropleth",
    data = gdf_blk_plt,
    columns=['GEOID20', 'count'],
    key_on="feature.properties.GEOID20",
    fill_color="BuPu",
    fill_opacity=0.7,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    legend_name="311 Requests per Block",
    bins=bins,
    reset = True
)

blk_map.add_child(choro_blk)

# block tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['name', 'councilId', 'GEOID20', 'count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block Census ID', '# of Requests'],
    localize = True
))

# NC boundary lines
folium.GeoJson(data = gdf_nc["geometry"], name = 'NC Boundaries',
               style_function = lambda x:{
                   "color": "#000000",
                   "weight": 1,
                   "opacity": .5,
                   "fill": False
               }
              ).add_to(blk_map)

folium.LayerControl().add_to(blk_map)

blk_map

# Type of Request by Census Block

In [ ]:
# create groupby neighborhood council
df_blk_type_ct = df_blk_nc.groupby(['councilNam', 'GEOID20', 'typeId'])['typeName'].value_counts().reset_index(name = 'type_count')
print(df_blk_type_ct.shape)
df_blk_type_ct.head()

In [ ]:
# merge block grouping with geometries
gdf_blk_type_plt = pd.merge(gdf_blk_nc, df_blk_type_ct, how = 'inner', on = 'GEOID20')
gdf_blk_type_plt.drop(columns = ['name'], inplace = True)
gdf_blk_type_plt.head()

In [ ]:
# block basemap
fig2 = folium.Figure(height = 850, width = 1050)
blk_type_map = folium.Map(location = (34.2, -118.39), tiles='cartodbpositron', zoom_start=9.5)
fig2.add_child(blk_type_map)

# block-by-block requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
bins = gdf_blk_type_plt["type_count"].quantile([0, 0.25, 0.5, 0.6, .7, .8, .9, .95, .98, 1]).tolist()


##### ANIMAL REMAINS
# Animal Remains requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
choro_blk_AR = folium.Choropleth(
    geo_data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Animal Remains'].to_json(),
    name="Animal Remains",
    data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Animal Remains'],
    columns=['GEOID20', 'type_count'],
    key_on="feature.properties.GEOID20",
    fill_color="YlGn",
    fill_opacity=1,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    bins = bins,
    legend_name="Animal Remains",
    show = False
    )

blk_type_map.add_child(choro_blk_AR)

# Block type tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk_AR.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['councilNam', 'councilId', 'GEOID20', 'typeName', 'type_count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block ID', 'Type of Request', '# of Requests'],
    localize = True
))


##### BULKY ITEMS
# Bulky Items requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
choro_blk_BI = folium.Choropleth(
    geo_data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Bulky Items'].to_json(),
    name="Bulky Items",
    data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Bulky Items'],
    columns=['GEOID20', 'type_count'],
    key_on="feature.properties.GEOID20",
    fill_color="OrRd",
    fill_opacity=1,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    bins = bins,
    legend_name="Bulky Items",
    show = False
    )

blk_type_map.add_child(choro_blk_BI)

# Block type tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk_BI.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['councilNam', 'councilId', 'GEOID20', 'typeName', 'type_count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block ID', 'Type of Request', '# of Requests'],
    localize = True
))


##### ELECTRONIC WASTE
# Electronic Waster requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
choro_blk_EW = folium.Choropleth(
    geo_data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Electronic Waste'].to_json(),
    name="Electronic Waste",
    data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Electronic Waste'],
    columns=['GEOID20', 'type_count'],
    key_on="feature.properties.GEOID20",
    fill_color="PuBuGn",
    fill_opacity=1,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    bins = bins,
    legend_name="Electronic Waste",
    show = False
    )

blk_type_map.add_child(choro_blk_EW)

# Block type tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk_EW.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['councilNam', 'councilId', 'GEOID20', 'typeName', 'type_count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block ID', 'Type of Request', '# of Requests'],
    localize = True
))


##### FEEDBACK
# Feedback requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
choro_blk_F = folium.Choropleth(
    geo_data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Feedback'].to_json(),
    name="Feedback",
    data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Feedback'],
    columns=['GEOID20', 'type_count'],
    key_on="feature.properties.GEOID20",
    fill_color="PuRd",
    fill_opacity=1,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    bins = bins,
    legend_name="Feedback",
    show = False
    )

blk_type_map.add_child(choro_blk_F)

# Block type tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk_F.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['councilNam', 'councilId', 'GEOID20', 'typeName', 'type_count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block ID', 'Type of Request', '# of Requests'],
    localize = True
))


##### GRAFFITI
# Graffiti requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
choro_blk_G = folium.Choropleth(
    geo_data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Graffiti'].to_json(),
    name="Graffiti",
    data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Graffiti'],
    columns=['GEOID20', 'type_count'],
    key_on="feature.properties.GEOID20",
    fill_color="YlGnBu",
    fill_opacity=1,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    bins = bins,
    legend_name="Graffiti",
    show = False
    )

blk_type_map.add_child(choro_blk_G)

# Block type tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk_G.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['councilNam', 'councilId', 'GEOID20', 'typeName', 'type_count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block ID', 'Type of Request', '# of Requests'],
    localize = True
))


##### HOMELESS ENCAMPMENT
# Homeless Encampment requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
choro_blk_HE = folium.Choropleth(
    geo_data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Homeless Encampment'].to_json(),
    name="Homeless Encampment",
    data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Homeless Encampment'],
    columns=['GEOID20', 'type_count'],
    key_on="feature.properties.GEOID20",
    fill_color="YlOrRd",
    fill_opacity=1,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    bins = bins,
    legend_name="Homeless Encampment",
    show = False
    )

blk_type_map.add_child(choro_blk_HE)

# Block type tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk_HE.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['councilNam', 'councilId', 'GEOID20', 'typeName', 'type_count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block ID', 'Type of Request', '# of Requests'],
    localize = True
))


##### ILLEGAL DUMPING
# Illegal Dumping requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
choro_blk_ID = folium.Choropleth(
    geo_data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Illegal Dumping'].to_json(),
    name="Illegal Dumping",
    data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Illegal Dumping'],
    columns=['GEOID20', 'type_count'],
    key_on="feature.properties.GEOID20",
    fill_color="RdPu",
    fill_opacity=1,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    bins = bins,
    legend_name="Illegal Dumping",
    show = False
    )

blk_type_map.add_child(choro_blk_ID)

# Block type tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk_ID.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['councilNam', 'councilId', 'GEOID20', 'typeName', 'type_count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block ID', 'Type of Request', '# of Requests'],
    localize = True
))


##### METAL / APPLIANCES
# Metal / Appliances requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
choro_blk_MA = folium.Choropleth(
    geo_data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Metal/Appliances'].to_json(),
    name="Metal / Appliances",
    data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Metal/Appliances'],
    columns=['GEOID20', 'type_count'],
    key_on="feature.properties.GEOID20",
    fill_color="Purples",
    fill_opacity=1,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    bins = bins,
    legend_name="Metal/Appliances",
    show = False
    )

blk_type_map.add_child(choro_blk_MA)

# Block type tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk_MA.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['councilNam', 'councilId', 'GEOID20', 'typeName', 'type_count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block ID', 'Type of Request', '# of Requests'],
    localize = True
))


##### MULTIPLE STREETLIGHTS
# Other requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
choro_blk_MS = folium.Choropleth(
    geo_data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Multiple Streetlights'].to_json(),
    name="Multiple Streetlights",
    data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Multiple Streetlights'],
    columns=['GEOID20', 'type_count'],
    key_on="feature.properties.GEOID20",
    fill_color="Greens",
    fill_opacity=1,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    bins = bins,
    legend_name="Multiple Streetlights",
    show = False
    )

blk_type_map.add_child(choro_blk_MS)

# Block type tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk_MS.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['councilNam', 'councilId', 'GEOID20', 'typeName', 'type_count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block ID', 'Type of Request', '# of Requests'],
    localize = True
))


##### OTHER
# Other requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
choro_blk_O = folium.Choropleth(
    geo_data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Other'].to_json(),
    name="Other",
    data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Other'],
    columns=['GEOID20', 'type_count'],
    key_on="feature.properties.GEOID20",
    fill_color="Greys",
    fill_opacity=1,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    bins = bins,
    legend_name="Other",
    show = False
    )

blk_type_map.add_child(choro_blk_O)

# Block type tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk_O.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['councilNam', 'councilId', 'GEOID20', 'typeName', 'type_count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block ID', 'Type of Request', '# of Requests'],
    localize = True
))


##### SINGLE STREETLIGHT
# Single Streetlight requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
choro_blk_SS = folium.Choropleth(
    geo_data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Single Streetlight'].to_json(),
    name="Single Street Light",
    data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Single Streetlight'],
    columns=['GEOID20', 'type_count'],
    key_on="feature.properties.GEOID20",
    fill_color="GnBu",
    fill_opacity=1,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    bins = bins,
    legend_name="Single Streetlight",
    show = False
    )

blk_type_map.add_child(choro_blk_SS)

# Block type tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk_SS.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['councilNam', 'councilId', 'GEOID20', 'typeName', 'type_count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block ID', 'Type of Request', '# of Requests'],
    localize = True
))


# WATER WASTE
# Water Waste requests choropleth
# adapted from https://python-visualization.github.io/folium/quickstart.html
choro_blk_WW = folium.Choropleth(
    geo_data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Water Waste'].to_json(),
    name="Water Waste",
    data = gdf_blk_type_plt[gdf_blk_type_plt['typeName'] == 'Water Waste'],
    columns=['GEOID20', 'type_count'],
    key_on="feature.properties.GEOID20",
    fill_color="BuPu",
    fill_opacity=1,
    nan_fill_opacity = 0,
    line_opacity=0.2,
    bins = bins,
    legend_name="Water Waste",
    show = False
    )

blk_type_map.add_child(choro_blk_WW)

# Block type tooltip
# adapted from https://stackoverflow.com/questions/61144877/currency-geojsontooltip-formatting-in-folium
choro_blk_WW.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['councilNam', 'councilId', 'GEOID20', 'typeName', 'type_count'],
    aliases = ['Neighborhood Council', 'Council ID', 'Block ID', 'Type of Request', '# of Requests'],
    localize = True
))


# NC boundary lines
folium.GeoJson(data = gdf_nc["geometry"], name = 'NC Boundaries', control = False,
               style_function = lambda x:{
                   "color": "#000000",
                   "weight": 1,
                   "opacity": .5,
                   "fill": False
               }
              ).add_to(blk_type_map)

folium.LayerControl(collapsed = False).add_to(blk_type_map)

blk_type_map

In [ ]:
# save type by nc
blk_type_map.save('../streamlit/blk_type_layered.html')